# Denison CS181/DA210 Homework 5.b - Step 1

Before you turn this notebook in, make sure everything runs as expected. This is a combination of **restarting the kernel** and then **running all cells**.

Make sure you fill in any place that says `# YOUR CODE HERE` or "YOUR ANSWER HERE".

---

## Part A

**Q1:** One of the reasons for the existence of *Unicode* is its ability to use strings that go beyond the limitations of the keyboard.  Relative to the discussion in the chapter, *Unicode* is about the *strings* we can use in our programs, and the issue of how they translate/map to a sequence of *bytes* (i.e. their *encoding*) is a separate concept.

When we have the *code-point* (generally a hex digit sequence identifying an index into the set of characters) for a Unicode character that is beyond our normal keyboard characters, we can include them in our strings by using the `\u` escape prefix followed by the hex digits for the code-point.  Consider the Python string s:

    s = "Unicode examples: \u2B2C and \u266A and \u1F60 and " \
        "\u265E and \u0394 and \u0402"
        
Write code to print `s`, then assign to `b8` the UTF-8 encoding of `s`, and `b16` the UTF-16BE encoding of `s`.  For each, use the `.hex()` method of the `bytes` data type to see a hex version of the encoded values.

In [4]:
s = "Unicode examples: \u2B2C and \u266A and \u1F60 and " \
    "\u265E and \u0394 and \u0402"
b8=s.encode()
b16=s.encode(encoding="UTF-16BE")

In [5]:
b8.hex()

'556e69636f6465206578616d706c65733a20e2acac20616e6420e299aa20616e6420e1bda020616e6420e2999e20616e6420ce9420616e6420d082'

In [6]:
b16.hex()

'0055006e00690063006f006400650020006500780061006d0070006c00650073003a00202b2c00200061006e00640020266a00200061006e006400201f6000200061006e00640020265e00200061006e00640020039400200061006e006400200402'

**Q2:** Consider the results of Q1, and answer the following questions: 

- which of the hex representations is longer?
- give explicit lengths for `b8`, `b16`, and for the two `hex()` transformations.
- how does this compare to the length of `s`?

In [7]:
print("Length of b16",len(b16))
print("Length of b8", len(b8))
print("Length of the hex of b8", len(b8.hex()))
print("Length of the hex of b16", len(b16.hex()))
print("Length of s", len(s))

Length of b16 98
Length of b8 59
Length of the hex of b8 118
Length of the hex of b16 196
Length of s 49


**Q3:** Write a function
```
    shiftLetter(letter, n)
```
whose parameter, `letter` should be a single character.  If the character is between `"A"` and `"Z"`, the function returns an upper case character $n$ positions further along, and "wrapping" if the + $n$ mapping goes past `"Z"`.  Likewise, it should map the lowercase characters between `"a"` and `"z"`.  If the parameter `letter` is anything else, or not of length 1, the function should return `letter`.

Hint: review functions `ord()` and `chr()` from Section 21.1.1, as well as the modulus operator `%`.

In [8]:
def shiftLetter(letter,n):
    if len(letter)>1 or ord("z")<ord(letter) or ord(letter)<ord("A") or ord("Z")<ord(letter)<ord("a"):
        return letter
    if ord(letter)<ord("a"):
        i=ord(letter)
        if i+n>ord("Z"):
            return chr(i+n-ord("Z")+ord("A")-1)
        return chr(i+n)
    if ord("Z")<ord(letter)<=ord("z"):
        i=ord(letter)
        if i+n>ord("z"):
            return chr(i+n-ord("z")+ord("a")-1)
        return chr(i+n)
    

In [9]:
# Debugging cell
print(shiftLetter("A", 1))
print(shiftLetter("Z", 1))
print(shiftLetter("M", 3))
print(shiftLetter("X", 2))

B
A
P
Z


In [10]:
# Testing cell
assert shiftLetter("A", 1) == "B"
assert shiftLetter("Z", 1) == "A"
assert shiftLetter("M", 3) == "P"
assert shiftLetter("X", 2) == "Z"

assert shiftLetter("a", 1) == "b"
assert shiftLetter("z", 1) == "a"
assert shiftLetter("m", 3) == "p"
assert shiftLetter("x", 2) == "z"


assert shiftLetter(" ", 1) == " "
assert shiftLetter("-", 1) == "-"
assert shiftLetter("/", 3) == "/"
assert shiftLetter("^", 5) == "^"

assert shiftLetter("abc", 4) == "abc"

**Q4:** Building on the previous exercise, write a function
```
    encrypt(plaintext, n)
```
that performs a `shiftLetter` for each of the letters in `plaintext` and accumulates and returns the resultant string.

In [11]:
def encrypt(plaintext,n):
    res=''
    for letter in plaintext:
        res+=shiftLetter(letter,n)
    return res

In [12]:
# Debugging cell
print(encrypt("abc", 1))
print(encrypt("xyz XYZ", 2))

bcd
zab ZAB


In [13]:
# Testing cell
assert encrypt("abc", 1) == "bcd"
assert encrypt("xyz XYZ", 2) == "zab ZAB"

**Q5:** Write a function 
```
    singleByteChars(s)
   ``` 
that takes its argument, `s`, and returns `True` if all the characters in `s` can be encoded by a single byte, and `False` otherwise.

In [23]:
def singleByteChars(s):
    for char in s:
        if ord(char)>256:
            return False
        return True

print(singleByteChars("Hello \u2B2C"))
singleByteChars("Hello")

True


True

In [15]:
# Testing cell
assert singleByteChars("Hello \u2B2C") == False
assert singleByteChars("Hello") == True

AssertionError: 

**Q6:** Suppose you have, in your Python program, a variable that refers to a `bytes` data type, like `mystery` refers to the `bytes` constant literal as given here:
```
    mystery = b'\xc9\xa2\x95}\xa3@\x89\xa3@\x87\x99\x85'    \
              b'\x81\xa3@\xa3\x96@\x82\x85@\xa2\x96\x93'    \
              b'\xa5\x89\x95\x87@\x97\x99\x96\x82\x93\x85'  \
              b'\x94\xa2o@@\xe8\x96\xa4@\x82\x85\xa3Z'
```              
Perhaps this value came from a network message, or from a file.  But you suspect that it, in fact, holds the bytes for a character string, and you need to figure out how it is encoding.  Assume that you have narrowed the encodings down to one of the following:

- 'UTF-8'
- 'UTF-16BE'
- 'cp037'
- 'latin_1'

Write code to convert the byte sequence to a character string, and determine the correct encoding.  By the end of your code, assign to `s_mystery` the correctly decoded string.

In [47]:
mystery = b'\xc9\xa2\x95}\xa3@\x89\xa3@\x87\x99\x85'    \
          b'\x81\xa3@\xa3\x96@\x82\x85@\xa2\x96\x93'    \
          b'\xa5\x89\x95\x87@\x97\x99\x96\x82\x93\x85'  \
          b'\x94\xa2o@@\xe8\x96\xa4@\x82\x85\xa3Z'
for method in ['UTF-8','UTF-16BE','cp037','latin_1']:
    try:
        s_mystery=mystery.decode(encoding=method)   
    except UnicodeDecodeError:
        continue
    for char in s_mystery:
        if ord(char)>127:
            break
    else:
        print(s_mystery)


Isn't it great to be solving problems?  You bet!


---

---

## Part B

**Q7:** How much time (in minutes/hours) did you spend on this homework assignment?

1 and a half hours

**Q8:** Who was your partner for this assignment?  If you worked alone, say so instead.

I worked alone